# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv (r'starter_code\city_data.csv')
print (df)

     Unnamed: 0               City    Lat     Lng  Max Temp  Humidity  \
0             0          Cape Town -33.93   18.42     62.01        87   
1             1  West Lake Stevens  47.99 -122.10     63.00        71   
2             2          Saint-Leu -21.15   55.28     73.40        69   
3             3       Punta Arenas -53.15  -70.92     48.20        61   
4             4        Yellowknife  62.46 -114.35     24.80        73   
..          ...                ...    ...     ...       ...       ...   
237         237          Dalbandin  28.89   64.41     52.47        36   
238         238               Abay  49.63   72.85     26.60        86   
239         239              Ginir   7.13   40.70     55.47        83   
240         240  Zheleznodorozhnyy  55.75   38.00     41.00       100   
241         241         Gualaquiza  -3.40  -78.55     84.51        45   

     Cloudiness  Wind Speed Country        Date  
0            75       16.11      ZA  1603919660  
1             1        

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
import gmaps.datasets

gmaps.configure(api_key='') # Fill in with your API key

In [5]:
locations = df[['Lat', 'Lng']]
weights = df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# weather_df = df.loc[(df['Max Temp'] >= 70,:) & (df['Max Temp'] <=80,:) & (df['Wind Speed'] <10]) & (df['Cloudiness']=0,:)
# weather_df

temp_first = df.loc[(df["Max Temp"] >= 68) & (df["Max Temp"] < 78), :]
wind_second = temp_first.loc[temp_first["Wind Speed"] < 10, :]
humidity_third = wind_second.loc[(wind_second["Humidity"] >= 30) & (wind_second["Humidity"] < 70), :]
cloudy_fourth = humidity_third.loc[humidity_third["Cloudiness"] < 70 ,:]
cloudy_fourth.count()

# got the count, then drop if anything appears and then create df

# clean_df = cloudy_fourth.drop('Unnamed:0')
clean_df = cloudy_fourth.dropna(how = 'any') 
clean_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Puerto Ayora,-0.74,-90.35,77.00,65,20,8.05,EC,1603919771
71,71,Porto União,-26.24,-51.08,74.62,41,0,3.44,BR,1603919865
91,91,Faranah,10.04,-10.74,73.08,63,0,1.97,GN,1603919867
112,112,Half Moon Bay,37.46,-122.43,73.99,32,1,8.05,US,1603919609
157,157,Castro,-24.79,-50.01,72.57,50,0,3.15,BR,1603919797
213,213,Kodīnar,20.79,70.70,76.98,58,0,7.76,IN,1603919879


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
# add hotel name column
hotel_df = clean_df
hotel_df['hotel name'] = ''
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel name
6,6,Puerto Ayora,-0.74,-90.35,77.00,65,20,8.05,EC,1603919771,
71,71,Porto União,-26.24,-51.08,74.62,41,0,3.44,BR,1603919865,
91,91,Faranah,10.04,-10.74,73.08,63,0,1.97,GN,1603919867,
112,112,Half Moon Bay,37.46,-122.43,73.99,32,1,8.05,US,1603919609,
157,157,Castro,-24.79,-50.01,72.57,50,0,3.15,BR,1603919797,
213,213,Kodīnar,20.79,70.70,76.98,58,0,7.76,IN,1603919879,


In [52]:
# hotel_df['Lat'].sum()

6.119999999999996

In [8]:
locations = hotel_df[["Lat", "Lng"]].astype(float)
hotels = "lodging"
radius = 5000
#print(locations)

#add parameters and define base url
params = {
    "location": locations,
    "type": hotels,
    "radius": radius,
    "key": g_key
}
                      
#insert for loop, look up iterrows
for index, row in hotel_df.iterrows():
# add keyword to params dict
#     params['Lat_location'] = row['Lat'].astype(float)
#     params['Lng_location'] = row['Lng'].astype(float)

    lat=row['Lat']
    lng=row['Lng']
    params['location']= f"{lat},{lng}"
                      
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
                      
#run request
    data = requests.get(base_url, params=params).json()
    
    try:
        hotel_name = data["results"][0]["name"]
        print(f"retrieving results for index {index}: {hotel_name}.")
    
        hotel_df.loc[index, 'hotel name'] = hotel_name
        # ^linked to 2 lines above
    
    except(KeyError, IndexError):
        print("cannotfind hotel for", row["City"])               

#look at nearest restaurant (plug and run) and airport activity in apis

retrieving results for index 6: Finch Bay Galapagos Hotel.
retrieving results for index 71: Hotel 10.
retrieving results for index 91: Chez mon oncle.
retrieving results for index 112: The Ritz-Carlton, Half Moon Bay.
retrieving results for index 157: CHACARA BAILLY.
retrieving results for index 213: Hotel Krishna Park, Kodinar.


In [9]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hotel name
6,6,Puerto Ayora,-0.74,-90.35,77.00,65,20,8.05,EC,1603919771,Finch Bay Galapagos Hotel
71,71,Porto União,-26.24,-51.08,74.62,41,0,3.44,BR,1603919865,Hotel 10
91,91,Faranah,10.04,-10.74,73.08,63,0,1.97,GN,1603919867,Chez mon oncle
112,112,Half Moon Bay,37.46,-122.43,73.99,32,1,8.05,US,1603919609,"The Ritz-Carlton, Half Moon Bay"
157,157,Castro,-24.79,-50.01,72.57,50,0,3.15,BR,1603919797,CHACARA BAILLY
213,213,Kodīnar,20.79,70.70,76.98,58,0,7.76,IN,1603919879,"Hotel Krishna Park, Kodinar"


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
# add as a layer

# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2 = gmaps.figure()
fig2.add_layer(gmaps.heatmap_layer(locations, weights=hotel_df['Humidity']))
fig2.add_layer(markers)
fig2

Figure(layout=FigureLayout(height='420px'))